# Biologics Pharmacology AI/ML Workflow Notebook

This notebook provides an interactive, end-to-end walkthrough for building and validating data-driven models to support early decisions in biologics drug development.

## What this notebook covers

1. Generate (or load) integrated synthetic biologics data
2. Validate schema and inspect key variables
3. Run grouped cross-validation by molecule family
4. Train a final multi-output model for PK, PD, and safety
5. Rank molecules with a transparent decision score
6. Visualize target distributions, CV metrics, and feature importance

> You can replace synthetic data with real preclinical/clinical data once available.

In [ ]:
from pathlib import Path
import sys

import joblib
import pandas as pd
from IPython.display import display

PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from biologics_pharmacology.decision_support import rank_candidates
from biologics_pharmacology.modeling import (
    run_grouped_cross_validation,
    train_final_model,
)
from biologics_pharmacology.schema import feature_columns, validate_dataset_schema
from scripts.generate_synthetic_biologics_data import generate_dataset

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

## 1) Load or generate biologics dataset

The code below will reuse `data/synthetic_biologics.csv` if it already exists. Otherwise, it generates a new synthetic dataset.

In [ ]:
data_path = Path("data/synthetic_biologics.csv")
data_path.parent.mkdir(parents=True, exist_ok=True)

if data_path.exists():
    df = pd.read_csv(data_path)
    source = "loaded from existing CSV"
else:
    df = generate_dataset(n_samples=500, seed=42)
    df.to_csv(data_path, index=False)
    source = "generated and saved"

validate_dataset_schema(df)

print(f"Dataset source: {source}")
print(f"Rows: {len(df)} | Columns: {len(df.columns)}")
print(f"Unique molecule families: {df['molecule_family'].nunique()}")

df.head()

## 2) Quick profile of key variables

This gives a high-level summary across modalities, therapeutic areas, and model targets.

In [ ]:
target_columns = [
    "clinical_pk_half_life_day",
    "clinical_pd_response_pct",
    "severe_ae_rate_pct",
]

print("Counts by modality:")
display(df["modality"].value_counts().to_frame("count"))

print("\nCounts by therapeutic area:")
display(df["therapeutic_area"].value_counts().to_frame("count"))

print("\nTarget summary:")
display(df[target_columns].describe().round(3))

## 3) Grouped cross-validation

We evaluate with `GroupKFold` on `molecule_family` to reduce leakage across related molecules.

In [ ]:
cv_metrics = run_grouped_cross_validation(df, n_splits=5)

cv_summary = (
    cv_metrics.groupby("target")[["mae", "r2"]]
    .agg(["mean", "std"])
    .round(3)
)

print("Fold-level sample:")
display(cv_metrics.head())

print("\nGrouped CV summary by target:")
display(cv_summary)

## 4) Train final model on all data

After validation, train a final model artifact for downstream decision support.

In [ ]:
artifacts = train_final_model(df)

model_path = Path("models/biologics_multitask_model.joblib")
model_path.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(artifacts, model_path)

print(f"Saved model artifacts to: {model_path}")

## 5) Rank candidate biologics for early development decisions

The score favors:
- higher predicted PD response
- longer predicted PK half-life
- lower predicted severe AE rate

In [ ]:
candidate_table = df[["molecule_id"] + feature_columns()].copy()

top_candidates = rank_candidates(
    artifacts=artifacts,
    candidates=candidate_table,
    top_k=15,
)

top_candidates[
    [
        "molecule_id",
        "pred_clinical_pk_half_life_day",
        "pred_clinical_pd_response_pct",
        "pred_severe_ae_rate_pct",
        "decision_score",
    ]
].round(3)

## 6) Visual analytics: targets, validation, and model behavior

These plots help communicate model behavior to cross-functional QP and development teams.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", context="notebook")

fig, axes = plt.subplots(1, 3, figsize=(18, 4))
colors = ["#4C78A8", "#59A14F", "#E15759"]

for idx, target in enumerate(target_columns):
    sns.histplot(df[target], kde=True, ax=axes[idx], color=colors[idx])
    axes[idx].set_title(f"Distribution: {target}")
    axes[idx].set_xlabel(target)

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(1, 3, figsize=(18, 4))
for idx, target in enumerate(target_columns):
    sns.boxplot(data=df, x="modality", y=target, ax=axes[idx], color="#A0CBE8")
    axes[idx].set_title(f"{target} by Modality")
    axes[idx].tick_params(axis="x", rotation=30)

plt.tight_layout()
plt.show()

### Cross-validation performance plots

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

sns.barplot(
    data=cv_metrics,
    x="target",
    y="mae",
    errorbar="sd",
    ax=axes[0],
    palette="Blues_d",
)
axes[0].set_title("Grouped CV MAE by Target")
axes[0].tick_params(axis="x", rotation=20)

sns.barplot(
    data=cv_metrics,
    x="target",
    y="r2",
    errorbar="sd",
    ax=axes[1],
    palette="Greens_d",
)
axes[1].set_title("Grouped CV R2 by Target")
axes[1].tick_params(axis="x", rotation=20)

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
sns.boxplot(data=cv_metrics, x="target", y="mae", ax=axes[0], palette="Blues")
axes[0].set_title("Fold Variability: MAE")
axes[0].tick_params(axis="x", rotation=20)

sns.boxplot(data=cv_metrics, x="target", y="r2", ax=axes[1], palette="Greens")
axes[1].set_title("Fold Variability: R2")
axes[1].tick_params(axis="x", rotation=20)

plt.tight_layout()
plt.show()

### Feature importance by target

Because the model is multi-output, we inspect importances separately for PK, PD, and safety.

In [ ]:
preprocessor = artifacts.pipeline.named_steps["preprocess"]
regressor = artifacts.pipeline.named_steps["regressor"]

feature_names = preprocessor.get_feature_names_out()

target_importance_tables = {}
fig, axes = plt.subplots(1, len(artifacts.target_columns), figsize=(20, 5))

for idx, (target, estimator) in enumerate(zip(artifacts.target_columns, regressor.estimators_)):
    importance_df = pd.DataFrame(
        {
            "feature": feature_names,
            "importance": estimator.feature_importances_,
        }
    ).sort_values("importance", ascending=False)

    target_importance_tables[target] = importance_df

    top_importance = importance_df.head(15).iloc[::-1]
    axes[idx].barh(top_importance["feature"], top_importance["importance"], color="#4C78A8")
    axes[idx].set_title(f"Top features: {target}")
    axes[idx].set_xlabel("Importance")
    axes[idx].tick_params(axis="y", labelsize=8)

plt.tight_layout()
plt.show()

for target in artifacts.target_columns:
    print(f"Top 10 features for {target}:")
    display(target_importance_tables[target].head(10).reset_index(drop=True))

### Decision landscape view

This plot shows how candidates balance efficacy, safety, and PK durability.

In [ ]:
scored_all = rank_candidates(
    artifacts=artifacts,
    candidates=candidate_table,
    top_k=len(candidate_table),
)

plt.figure(figsize=(10, 7))
scatter = sns.scatterplot(
    data=scored_all,
    x="pred_clinical_pd_response_pct",
    y="pred_severe_ae_rate_pct",
    size="pred_clinical_pk_half_life_day",
    hue="decision_score",
    palette="viridis",
    sizes=(20, 220),
    alpha=0.75,
)
scatter.set_title("Candidate decision landscape")
scatter.set_xlabel("Predicted clinical PD response (%)")
scatter.set_ylabel("Predicted severe AE rate (%)")

for _, row in scored_all.head(5).iterrows():
    plt.annotate(
        row["molecule_id"],
        (row["pred_clinical_pd_response_pct"], row["pred_severe_ae_rate_pct"]),
        xytext=(5, 5),
        textcoords="offset points",
        fontsize=9,
    )

plt.tight_layout()
plt.show()

scored_all[
    [
        "molecule_id",
        "decision_score",
        "pred_clinical_pd_response_pct",
        "pred_clinical_pk_half_life_day",
        "pred_severe_ae_rate_pct",
    ]
].head(10).round(3)

## 7) Adapting this notebook to real R&D data

To move from synthetic to real biologics programs:

1. Replace `df` with your governed integrated dataset.
2. Keep required schema columns from `biologics_pharmacology/schema.py`.
3. Preserve grouped validation by molecule family (or program-level grouping).
4. Tune model settings and decision-score weights with QP and clinical stakeholders.
5. Add uncertainty estimates before deployment into portfolio decisions.